# Import , Set working directory

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re
import time
import shap

from scipy import stats
from scipy.stats import randint

from imblearn.over_sampling import SMOTE
from boruta import BorutaPy

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import ElasticNet

from sklearn.naive_bayes import BernoulliNB
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import RFE
from sklearn.tree import export_graphviz
from sklearn.svm import SVC
from sklearn.svm import SVR
from sklearn.linear_model import LassoCV
from sklearn.datasets import make_regression

from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import make_scorer
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve 

%matplotlib inline

import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

import warnings
warnings.filterwarnings('ignore')

# import all our project classes
from magene import *

In [2]:
print("Current Working Directory " , os.getcwd())

if os.path.exists("C:/Users/micha.DESKTOP-8HA2IGV/OneDrive/Programming/Propulsion Project/intelligencia_backup/intelligencia") :
    # Change the current working Directory    
    os.chdir("C:/Users/micha.DESKTOP-8HA2IGV/OneDrive/Programming/Propulsion Project/intelligencia_backup/intelligencia")
    print("New Working Directory " , os.getcwd())
else:
    print("Can't change the Current Working Directory")    
    print("Current Working Directory " , os.getcwd())

Current Working Directory  /Users/luciepieckova/Desktop/PA_project/intelligencia
Can't change the Current Working Directory
Current Working Directory  /Users/luciepieckova/Desktop/PA_project/intelligencia


Preprocess a provided CSV file to smaller chunks with relevant data for our analysis.

# Data Exploration

## Data Acquisition and Exploration

- Xena platform collects gene expression data from different studies and combines them in clean, normalized datasets. Data that was used for this project includes phenotype and genotype information about 19131 samples (they update the data regularly - so this number might increase) with various tissue types.

https://xenabrowser.net/datapages/?cohort=TCGA%20TARGET%20GTEx&removeHub=https%3A%2F%2Fxena.tree
- Genotype:

https://xenabrowser.net/datapages/?dataset=TcgaTargetGtex_rsem_gene_tpm&host=https%3A%2F%2Ftoil.xenahubs.net&removeHub=https%3A%2F%2Fxena.treehouse.gi.ucsc.edu%3A443
- Phenotype:

https://xenabrowser.net/datapages/?dataset=TcgaTargetGTEX_phenotype.txt&host=https%3A%2F%2Ftoil.xenahubs.net&removeHub=https%3A%2F%2Fxena.treehouse.gi.ucsc.edu%3A443


In [3]:
# Take a look at phenotype information which will be used to seperate the genotype information into individual chunks.
phenotype_path = 'Data/TcgaTargetGTEX_phenotype.txt'
df_phenotype = pd.read_csv(phenotype_path, '\t')
df_phenotype.head()

,sample,detailed_category,primary disease or tissue,_primary_site,_sample_type,_gender,_study
0,TCGA-V4-A9EE-01,Uveal Melanoma,Uveal Melanoma,Eye,Primary Tumor,Male,TCGA
1,TCGA-VD-AA8N-01,Uveal Melanoma,Uveal Melanoma,Eye,Primary Tumor,Male,TCGA
2,TCGA-V4-A9EI-01,Uveal Melanoma,Uveal Melanoma,Eye,Primary Tumor,Male,TCGA
3,TCGA-VD-AA8O-01,Uveal Melanoma,Uveal Melanoma,Eye,Primary Tumor,Male,TCGA
4,TCGA-WC-A888-01,Uveal Melanoma,Uveal Melanoma,Eye,Primary Tumor,Male,TCGA


In [4]:
# There are 19131 samples and six features with information about the samples.
df_phenotype.shape

(19131, 7)

In [5]:
# Genes are represented in Ensebml notation with information about the version of natation.
genotype_path = 'Data/TcgaTargetGtex_rsem_gene_tpm.txt'
df = pd.read_csv(genotype_path, delimiter='\t', usecols=['sample'])
df.head()

,sample
0,ENSG00000242268.2
1,ENSG00000259041.1
2,ENSG00000270112.3
3,ENSG00000167578.16
4,ENSG00000278814.1


In [6]:
# The data includes gene expression values for 60498 genes.
df.shape

(60498, 1)

# Data Preprocessing

## Create and Store Filtered Chunks 

Because drug development mainly focuses on protein-coding genes and in order to reduce dimensions the data is filtered tor protein-coding genes only.

In [7]:
# Define unique protein-coding genes.
def create_upcg():
    df_mart = pd.read_csv('Data/mart_export.txt')
    return set(df_mart[df_mart['Gene type'] == 'protein_coding']['Gene stable ID'].unique())

In [8]:
# Instantiate DataFilter class with paths of data and upcg function.
data_filter = DataFilter(genotype_path, phenotype_path, create_upcg())

In [9]:
# Create chunks by applying the split function. Store results as a csv.

data_filter.split(["Lung"], categories=["Lung Squamous Cell Carcinoma", "Lung"])
data_filter.split(["Lung"], categories=["Lung Adenocarcinoma", "Lung"])
data_filter.split(["Thyroid", "Thyroid Gland"])
data_filter.split(["Colon"])
data_filter.split(["Skin"])
data_filter.split(["Breast"], genders=["Female"])

Processing data with 787 samples.
Finished in 2.9 min

Processing data with 802 samples.
Finished in 3.1 min

Processing data with 784 samples.
Finished in 2.9 min

Processing data with 595 samples.
Finished in 2.6 min

Processing data with 659 samples.
Finished in 2.7 min

Processing data with 1160 samples.
Finished in 4.0 min



sample,label,ENSG00000167578.16,ENSG00000078237.5,ENSG00000146083.11,ENSG00000158486.13,ENSG00000198242.13,ENSG00000134108.12,ENSG00000172137.18,ENSG00000276644.4,ENSG00000094963.13,...,ENSG00000107863.16,ENSG00000213782.7,ENSG00000146707.14,ENSG00000158417.10,ENSG00000089177.17,ENSG00000186115.12,ENSG00000009694.13,ENSG00000123685.8,ENSG00000105063.18,ENSG00000181518.3
TCGA-C8-A1HL-01,1,4.978200,2.662400,3.958000,-0.375200,10.787700,5.741500,0.62390,3.345000,-0.19930,...,3.225100,4.901600,2.47270,5.651100,5.109400,-9.965800,-6.506400,1.02930,4.409500,-9.965800
TCGA-EW-A2FS-01,1,5.703500,1.269600,4.218900,-4.293400,10.146000,5.780100,1.40110,4.053200,2.31930,...,3.586300,4.932700,3.15560,5.851000,4.427700,-6.506400,-3.046900,1.46000,4.408100,-9.965800
TCGA-B6-A402-01,1,4.125200,1.623400,5.018000,-2.826200,9.893500,4.892900,2.97290,-3.625900,6.06030,...,5.110700,5.301300,3.98010,5.859200,3.166900,-9.965800,-5.011600,2.50610,4.449000,-9.965800
TCGA-A2-A3XX-01,1,4.873400,1.599800,4.185900,-2.114000,9.890400,4.724700,9.48210,-3.458000,5.21260,...,4.167600,5.116900,4.42230,5.719500,3.039300,-6.506400,-5.573500,2.07070,4.107800,-9.965800
TCGA-Z7-A8R5-01,1,5.497300,1.384600,3.301700,-5.573500,11.139800,4.665100,3.20800,1.460000,2.85220,...,2.611400,4.717600,4.08750,3.657800,2.150900,-9.965800,-4.608200,2.85220,4.490000,-9.965800
TCGA-D8-A1JU-01,1,5.445600,2.967400,3.944000,-6.506400,10.541600,6.397600,3.89930,4.521100,3.99740,...,4.839000,5.496000,2.82400,5.763400,3.846100,-1.937900,-1.117200,2.20190,5.279500,-9.965800
TCGA-B6-A0RL-01,1,5.034300,1.245500,4.396500,-5.573500,10.410900,5.733900,2.17340,2.208200,-3.81600,...,4.129400,5.763200,4.22890,5.584100,3.114500,-6.506400,-5.573500,-1.28280,5.951000,-9.965800
GTEX-X4EP-2926-SM-3P5YQ,0,5.121100,1.305100,5.353700,-3.171400,10.689900,5.683700,2.55360,1.144700,6.28710,...,4.896300,5.919100,4.32420,5.550300,2.485700,-9.965800,-0.027700,3.89830,4.692700,-9.965800
TCGA-E9-A3HO-01,1,5.645000,0.651700,4.541100,-0.150400,10.703700,5.240700,0.13880,2.239100,1.79950,...,3.682700,4.350600,3.06020,5.896300,3.144200,-5.573500,-5.573500,0.85680,4.947200,-9.965800
TCGA-BH-A0B0-01,1,5.200500,2.419800,3.768800,-2.114000,10.183500,5.383400,4.19700,3.254100,1.64200,...,4.138400,5.222300,4.50790,5.689300,3.320600,-2.314700,-3.816000,1.70090,4.841500,-9.965800


## Create Other Useful Chunks

### Create: All Sites, Cancerous and Healthy (12 labels)

In [12]:
chunks = [
    "Output/Chunk_LungSquamousCellCarcinoma_Lung.csv",
    "Output/Chunk_LungAdenocarcinoma_Lung.csv",
    "Output/Chunk_Thyroid_ThyroidGland.csv",
    "Output/Chunk_Colon.csv",
    "Output/Chunk_Skin.csv",
    "Output/Chunk_Breast.csv"
]

cancers = [
    "lung_s",
    "lung_a",
    "thyroid",
    "colon",
    "skin",
    "breast"
]

In [13]:
# Create a dataframe with all chunks. Healthy tissue will be labeled with 0, cancerous with 1.
chunk_df = pd.DataFrame()
for chunk, cancer in zip(chunks, cancers):
    # Load chunks.
    chunk = pd.read_csv(chunk)
    chunk.index = chunk.iloc[:,0]
    chunk.drop(columns = "Unnamed: 0", inplace = True)
    chunk.columns = [(re.sub('\.\d+', '', gene)) for gene in chunk.columns]
    
    # Label chunks according to sample type.
    chunk["label"].replace(1, "1_" + cancer, inplace = True)
    chunk["label"].replace(0, "0_" + cancer, inplace = True)
    
    # Append to complete chunk.
    chunk_df = chunk_df.append(chunk)

In [14]:
chunk_df.to_csv("Output/Chunk_AllCancers.csv")

### Create: All Sites, Only Cancerous

In [15]:
chunk_df = pd.read_csv("Output/Chunk_AllCancers.csv", index_col="Unnamed: 0")
chunk_cancer = chunk_df[chunk_df["label"].str.contains("1")]
chunk_cancer.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2982 entries, TCGA-56-8305-01 to TCGA-A2-A1FV-01
Columns: 19664 entries, label to ENSG00000181518
dtypes: float64(19663), object(1)
memory usage: 447.4+ MB


In [16]:
chunk_cancer.to_csv("Output/Chunk_AllCancers_1only.csv")

### Create: All Sites, Cancerous and Healthy (2 labels)

In [18]:
chunk_df_all = chunk_df.copy()
chunk_df_all['label'] = chunk_df_all['label'].astype(str).str[0]

In [19]:
chunk_df_all.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4781 entries, GTEX-1399S-1726-SM-5L3DI to GTEX-1117F-2826-SM-5GZXL
Columns: 19664 entries, label to ENSG00000181518
dtypes: float64(19663), object(1)
memory usage: 717.3+ MB


In [20]:
chunk_df_all.to_csv("Output/Chunk_AllCancers_0vs1.csv")

### Create: Lung_A Cancerous and Lung_S Cancerous

In [21]:
Chunk_LungA1_vs_LungS1 = chunk_df[chunk_df["label"].str.contains("1_lung")]
# Chunk_LungA1_vs_LungS1.info()
Chunk_LungA1_vs_LungS1["label"] = Chunk_LungA1_vs_LungS1["label"].replace("1_lung_s", "0")
Chunk_LungA1_vs_LungS1["label"] = Chunk_LungA1_vs_LungS1['label'].astype(str).str[0]

In [22]:
Chunk_LungA1_vs_LungS1.to_csv("Output/Chunk_LungA1_vs_LungS1.csv")
#1_lung_a = 1
#1_lung_s = 0

# Gene Selection

1. Now the data has been seperated into chunks and relevant genes can be selected.
- Five different models will be used to give each gene a feature importance.
- About 50 genes per sample type will be selected according to three criteria: Feature importance score, number of picks and overlaps with cosmic genes (platform which gives a list of tested cancer-associated genes):
https://cancer.sanger.ac.uk/cosmic
- These will all be combined in one dataframe.

## Select Top 400 Genes per Chunks

- Select top features in each sample type.
- Show model accuracies.
- Store results as csvs.

In [3]:
path_list = ["Output/Chunk_LungAdenocarcinoma_Lung.csv",
             "Output/Chunk_LungSquamousCellCarcinoma_Lung.csv",
             "Output/Chunk_Breast.csv",
             "Output/Chunk_Skin.csv",
             "Output/Chunk_Thyroid_ThyroidGland.csv",
             "Output/Chunk_LungA1_vs_LungS1.csv",
             "Output/Chunk_AllCancers_0vs1.csv",
             "Output/Chunk_Colon.csv"]
path_intogen_list = ["Data/Reference_Data/Census_allWed May 15 09_46_55 2019.csv"] * len(path_list)

evaluation = Evaluation(1888)
evaluation.iterate_trough_cancers(path_list, path_intogen_list, nrows=None, usecols=None, threshold=1)

Evaluating Output/Chunk_LungAdenocarcinoma_Lung.csv
Accuracy of GBM: 1.000
Accuracy of RFE: 0.971
Accuracy of Elastic Net: 0.969
Accuracy of Boruta: 0.999 (+/- 0.006)
Accuracy of Boruta Tree: 1.000
Accuracy of LassoCV 0.972
Finished in 6.3 min

Evaluating Output/Chunk_LungSquamousCellCarcinoma_Lung.csv
Accuracy of GBM: 0.996
Accuracy of RFE: 0.971
Accuracy of Elastic Net: 0.976
Accuracy of Boruta: 1.000 (+/- 0.000)
Accuracy of Boruta Tree: 0.992
Accuracy of LassoCV 0.976
Finished in 7.9 min

Evaluating Output/Chunk_Breast.csv
Accuracy of GBM: 0.997
Accuracy of RFE: 0.928
Accuracy of Elastic Net: 0.950
Accuracy of Boruta: 1.000 (+/- 0.000)
Accuracy of Boruta Tree: 1.000
Accuracy of LassoCV 0.957
Finished in 11.1 min

Evaluating Output/Chunk_Skin.csv
Accuracy of GBM: 0.990
Accuracy of RFE: 0.980
Accuracy of Elastic Net: 0.981
Accuracy of Boruta: 1.000 (+/- 0.000)
Accuracy of Boruta Tree: 1.000
Accuracy of LassoCV 0.981
Finished in 4.6 min

Evaluating Output/Chunk_Thyroid_ThyroidGland.csv

## Select Top 70-90 Genes per Chunk and Combine in one Dataframe

- Select about 70-90 genes with three criteria
    - Top 30 genes sorted by feature importance score (across models).
    - Top 30 genes sorted by count of picks (across models).
    - Top 30 genes sorted by model pick overlaps with cosmic genes.

In [3]:
fr = FilterResults(1888)

names = ["Skin", "Thyroid_ThyroidGland", "Colon", "Breast",
         "LungAdenocarcinoma_Lung", "LungSquamousCellCarcinoma_Lung",
         "AllCancers_0vs1"]
chunks = ["skin", "thyroid", "colon", "breast", "lung_a", "lung_s", "all"]
results = [pd.read_csv(f'Output/Result_{name}.csv', index_col=0) for name in names]

final_top_genes = fr.charmander(results, chunks)
final_top_genes.to_csv("Output/final_top_genes.csv")

skin :  69 	genes selected | 12 duplicates removed 	T: 19 ,I: 23 ,C: 16 ,TI: 6 ,TC: 4 ,IC: 0 ,TIC: 1
thyroid :  80 	genes selected | 4 duplicates removed 	T: 27 ,I: 29 ,C: 20 ,TI: 0 ,TC: 3 ,IC: 1 ,TIC: 0
colon :  81 	genes selected | 9 duplicates removed 	T: 22 ,I: 23 ,C: 28 ,TI: 6 ,TC: 1 ,IC: 0 ,TIC: 1
breast :  74 	genes selected | 7 duplicates removed 	T: 23 ,I: 23 ,C: 21 ,TI: 7 ,TC: 0 ,IC: 0 ,TIC: 0
lung_a :  81 	genes selected | 7 duplicates removed 	T: 25 ,I: 24 ,C: 25 ,TI: 4 ,TC: 1 ,IC: 2 ,TIC: 0
lung_s :  84 	genes selected | 6 duplicates removed 	T: 26 ,I: 26 ,C: 27 ,TI: 2 ,TC: 1 ,IC: 1 ,TIC: 1
all :  77 	genes selected | 13 duplicates removed 	T: 18 ,I: 21 ,C: 26 ,TI: 8 ,TC: 3 ,IC: 0 ,TIC: 1
Top genes overall:  419 genes selected | 127 duplicates removed


# Gene Importance

## Apply models get combined importance

In [2]:
fdf = FinalDataFrame(1888, "Output/final_top_genes.csv")

In [2]:
# don't run if you have all_cancers_model.sav already saved,
# which is not the case when you are running this for the first time
X_train, y_train, X_test, y_test = fdf.load_data("Output/Chunk_AllCancers.csv")
fdf.run_store_model(X_train, y_train, X_test, y_test, 'Output/all_cancers_model.sav')

In [ ]:
chunk_path = "Output/Chunk_AllCancers.csv"
filename = "Output/all_cancers_model.sav"
final_top_genes_path = "Output/final_top_genes.csv"
save_to_path = "Output/final_top_genes.csv"
df_final = fdf.squirtle(chunk_path, filename, final_top_genes_path, save_to_path, top_genes_n=50)

In [8]:
loaded_model = pickle.load(open(filename, 'rb'))

In [5]:
extract = ExtractExpression()

In [ ]:
save_to_path = "Output/expression_data_all_final_symbol_50.csv"
extract.final_genes_expression_data(chunk_path, all_final_path, save_to_path)

In [15]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1200 entries, 400 to 237
Data columns (total 5 columns):
Gene                1200 non-null object
Shap_Importance     1200 non-null object
Cancer              1200 non-null object
Combination         1200 non-null object
SHAP_Combination    1200 non-null object
dtypes: object(5)
memory usage: 56.2+ KB


In [19]:
df_final.to_csv("Output/final_genes_50.csv")

In [ ]:
top_genes = pd.read_csv("Output/final_top_genes.csv")
top_genes.head()